In [ ]:
!gcloud services enable aiplatform.googleapis.com

In [ ]:
%%bash
gcloud compute firewall-rules create allow-imc-streamlit \
    --direction=INGRESS \
    --priority=1000 \
    --network=default \
    --action=ALLOW \
    --rules=tcp:8501 \
    --source-ranges=0.0.0.0/0 2> /dev/null

gcloud compute firewall-rules list --filter name='allow-imc-streamlit' 2> /dev/null

NAME                 NETWORK  DIRECTION  PRIORITY  ALLOW     DENY  DISABLED
allow-imc-streamlit  default  INGRESS    1000      tcp:8501        False


In [ ]:
!pip install --quiet streamlit

In [ ]:
%%writefile gemini-chat.py
import streamlit as st
import json
import vertexai, google.auth
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part

def numero_habitantes(pais: str):
    if pais == "Polonia":
        return {"pais": "Polonia", "habitantes": 38_750_000}
    elif pais == "Ucrania":
        return {"pais": "Ucrania", "habitantes": 35_660_000}
    elif pais == "Alemania":
        return {"pais": "Alemania", "habitantes": 84_120_000}
    else:
        return {"error": "Valor no encontrado"}

herramientas_mapeo = {"numero_habitantes" : numero_habitantes}

# --- Generar Modelo
credentials, project_id = google.auth.default()
vertexai.init(project=project_id, location="us-central1")

context = """
You are an expert in geography, history and politics of Central Europe.
You can answer general questions, but when population data is needed,
you must call the function numero_habitantes. When asked about other countries,
say you are not qualified to answer questions about other countries.
"""
model_name = "gemini-2.5-flash-lite"
model = GenerativeModel(model_name, system_instruction = context)


# --- Web UI con Streamlit
st.title("Chat con Gemini")
prompt = st.text_input("Ask me about Central Europe")
if st.button("Submit") and prompt.strip():
    with st.chat_message("user"):
        st.write(prompt)
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            try:
    response = st.session_state.chat.send_message(prompt)
    part = response.candidates[0].content.parts[0]
    if part.function_call:
        fn_name = part.function_call.name
        fn_args = json.loads(part.function_call.arguments)

        st.info(f"🛠️ El agente está usando la herramienta: `{fn_name}`")
        if fn_name in herramientas_mapeo:
            fn_result = herramientas_mapeo[fn_name](**fn_args)

            response = st.session_state.chat.send_message(
                Part.from_function_call_result(
                    name=fn_name,
                    response=json.dumps(fn_result)
                )
            )

  st.write(response.candidates[0].content.parts[0].text)
 except Exception as e:
                    st.error(f"Error en la lógica del agente: {str(e)}")

Overwriting gemini-chat.py


In [ ]:
!streamlit run gemini-chat.py --server.port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.128.0.2:8501
  External URL: http://35.192.117.78:8501

/opt/conda/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
